In [2]:
import pandas as pd
import os
import joblib
from sklearn.preprocessing import OneHotEncoder

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI (Anti-Errore)
# ==============================================================================
CURRENT_DIR = os.getcwd()
# Se siamo in una sottocartella, risaliamo alla root
if os.path.basename(CURRENT_DIR) in ["02_feature_engineering", "notebooks", "src"]:
    BASE_DIR = os.path.dirname(CURRENT_DIR)
else:
    BASE_DIR = CURRENT_DIR

DATA_DIR = os.path.join(BASE_DIR, 'dataset')
MODEL_DIR = os.path.join(BASE_DIR, 'models')

# Input: I file puliti dallo Split
TRAIN_FILE = os.path.join(DATA_DIR, 'train_final.csv')
VAL_FILE = os.path.join(DATA_DIR, 'validation.csv')
TEST_FILE = os.path.join(DATA_DIR, 'test.csv')

# Output: File "Encoded" (Solo categorie trasformate)
TRAIN_OUT = os.path.join(DATA_DIR, 'train_encoded.csv')
VAL_OUT = os.path.join(DATA_DIR, 'val_encoded.csv')
TEST_OUT = os.path.join(DATA_DIR, 'test_encoded.csv')

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# Verifica
if not os.path.exists(TRAIN_FILE):
    raise FileNotFoundError(f"❌ Errore: Non trovo {TRAIN_FILE}")

# ==============================================================================
# 2. CARICAMENTO
# ==============================================================================
print("📥 Caricamento dataset...")
df_train = pd.read_csv(TRAIN_FILE)
df_val = pd.read_csv(VAL_FILE)
df_test = pd.read_csv(TEST_FILE)

# Colonne da toccare (Solo categorie)
cat_cols = ['SEX', 'VAX_MANU']

# ==============================================================================
# 3. SOLO ONE-HOT ENCODING
# ==============================================================================
print("🔠 Esecuzione One-Hot Encoding (Sesso, Produttori)...")

# Configurazione Encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=int)

# A. FIT (Impara le categorie solo dal Train)
encoder.fit(df_train[cat_cols])

# Nomi nuove colonne
new_col_names = encoder.get_feature_names_out(cat_cols)
print(f"   -> Nuove colonne generate: {list(new_col_names)}")

# Funzione per applicare l'encoding mantenendo il resto invariato
def apply_encoding_only(df, enc, cols_to_encode, new_names):
    # 1. Trasforma le categorie in numeri
    matrix = enc.transform(df[cols_to_encode])
    # 2. Crea DataFrame temporaneo
    df_encoded = pd.DataFrame(matrix, columns=new_names, index=df.index)
    # 3. Unisci: Togli le vecchie categorie, metti le nuove.
    #    NOTA: I numeri (Età, Giorni) restano intatti nel 'df.drop'.
    df_final = pd.concat([df.drop(columns=cols_to_encode), df_encoded], axis=1)
    return df_final

# B. TRANSFORM (Applica a tutti)
df_train = apply_encoding_only(df_train, encoder, cat_cols, new_col_names)
df_val = apply_encoding_only(df_val, encoder, cat_cols, new_col_names)
df_test = apply_encoding_only(df_test, encoder, cat_cols, new_col_names)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
print(f"\n💾 Salvataggio file ENCODED (No Scaling) in: {DATA_DIR}")
df_train.to_csv(TRAIN_OUT, index=False)
df_val.to_csv(VAL_OUT, index=False)
df_test.to_csv(TEST_OUT, index=False)

# Salviamo solo l'encoder (Lo scaler non esiste in questa versione)
joblib.dump(encoder, os.path.join(MODEL_DIR, 'encoder_cat.pkl'))

print("-" * 40)
print("✅ OPERAZIONE COMPLETATA (Solo Encoding)")
print("-" * 40)
print("Esempio dati (Notare l'età originale e le nuove colonne binarie):")
# Mostriamo Età (numerica pura) e una colonna binaria nuova
cols_preview = ['AGE_YRS', 'NUMDAYS'] + list(new_col_names)[:2]
print(df_train[cols_preview].head(3))

📥 Caricamento dataset...
🔠 Esecuzione One-Hot Encoding (Sesso, Produttori)...
   -> Nuove colonne generate: ['SEX_F', 'SEX_M', 'VAX_MANU_JANSSEN', 'VAX_MANU_MODERNA', 'VAX_MANU_NOVAVAX', 'VAX_MANU_OTHER', 'VAX_MANU_PFIZER']

💾 Salvataggio file ENCODED (No Scaling) in: /Users/marcodonatiello/PycharmProjects/JupyterProject/dataset
----------------------------------------
✅ OPERAZIONE COMPLETATA (Solo Encoding)
----------------------------------------
Esempio dati (Notare l'età originale e le nuove colonne binarie):
   AGE_YRS  NUMDAYS  SEX_F  SEX_M
0       74      0.0      0      1
1       81    107.0      0      1
2       61      0.0      1      0
